# 1 часть.
### Цель задания – познакомиться со структурой данных ДДП.

#### импорты

In [ ]:
from random import randint
import time
import pandas as pd

#### создание класса "дерево"

In [ ]:
# Узел
class Node:
    def __init__(self, key):
        self.left = None
        self.right = None
        self.val = key
        self.parent = None


# Дерево
class BinarySearchTree:
    def __init__(self):
        self.root = None

    # Добавление узла
    def insert(self, key):
        if not self.root:
            self.root = Node(key)
            self.root.parent = Node(None)
        else:
            self.find_a_place_to_insert(self.root, key)

    def find_a_place_to_insert(self, node, key):  # рекурсивный поиск места для вставки
        if key < node.val:
            if node.left is None:
                node.left = Node(key)
                node.left.parent = node
            else:
                self.find_a_place_to_insert(node.left, key)
        elif key >= node.val:
            if node.right is None:
                node.right = Node(key)
                node.right.parent = node
            else:
                self.find_a_place_to_insert(node.right, key)


    # Удаление узла
    def delete(self, key):
        self.root = self.find_the_element_for_delete(self.root, key)

    def find_the_element_for_delete(self, root, key):
        if root is None:
            return root
        if key < root.val:
            root.left = self.find_the_element_for_delete(root.left, key)
        elif key > root.val:
            root.right = self.find_the_element_for_delete(root.right, key)
        else:
            if root.left is None:
                return root.right
            elif root.right is None:
                return root.left
            root.val = self.min_value_node(root.right).val
            root.right = self.find_the_element_for_delete(root.right, root.val)
        return root

    def min_value_node(self, node):
        current = node
        while(current.left is not None):
            current = current.left
        return current

    # Поиск по дереву (возвращает ссылку на вершину)
    def search(self, key):
        return self.find_the_element(self.root, key)

    def find_the_element(self, root, key):
        if root is None or root.val == key:
            return root
        if root.val < key:
            return self.find_the_element(root.right, key)
        return self.find_the_element(root.left, key)


    # Вывод дерева
    def create_graph(self):
        G = nx.Graph()
        self.networkx_tree(self.root, G)
        return G

    def networkx_tree(self, node, G):
        if node is not None:
            G.add_node(node.val)
            if node.left is not None:
                G.add_edge(node.val, node.left.val)
                self.networkx_tree(node.left, G)
            if node.right is not None:
                G.add_edge(node.val, node.right.val)
                self.networkx_tree(node.right, G)

    def display(self):
        lines, *_ = self.display_aux(self.root)
        for line in lines:
            print(line)

    def display_aux(self, node):
        # Если нет потомков
        if node is None:
            return [], 0, 0, 0
        if node.right is None and node.left is None:
            line = "%s" % node.val
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        # Только один потомок слева
        if node.right is None:
            lines, n, p, x = self.display_aux(node.left)
            s = "%s" % node.val
            u = len(s)
            first_line = (x + 1) * " " + (n - x - 1) * "_" + s
            second_line = x * " " + "/" + (n - x - 1 + u) * " "
            shifted_lines = [line + u * " " for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

        # Только один потомок справа
        if node.left is None:
            lines, n, p, x = self.display_aux(node.right)
            s = "%s" % node.val
            u = len(s)
            first_line = s + x * "_" + (n - x) * " "
            second_line = (u + x) * " " + "\\" + (n - x - 1) * " "
            shifted_lines = [u * " " + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        # Два потомка
        left, n, p, x = self.display_aux(node.left)
        right, m, q, y = self.display_aux(node.right)
        s = "%s" % node.val
        u = len(s)
        first_line = (x + 1) * " " + (n - x - 1) * "_" + s + y * "_" + (m - y) * " "
        second_line = (
            x * " " + "/" + (n - x - 1 + u + y) * " " + "\\" + (m - y - 1) * " "
        )
        if p < q:
            left += [n * " "] * (q - p)
        elif q < p:
            right += [m * " "] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * " " + b for a, b in zipped_lines]

        return lines, n + m + u, max(p, q) + 2, n + u // 2

    # для сортировки
    def find_parent(self, key):
        return self.search(key).val

    def max_value_node(self, node):
        current = node
        while(current.right is not None):
            current = current.right
        return current



#### пример использования класса

In [ ]:
bst = BinarySearchTree()

print("Добавляем элементы в дерево")
bst.insert(50)
bst.insert(30)
bst.insert(20)
bst.insert(40)
bst.insert(70)
bst.insert(60)
bst.insert(80)

print("Выводим дерево")
bst.display()

print("\nУдаляем элемент")
bst.delete(20)
bst.display()

print("\nПоиск элемента")
print(bst.search(40).val)



Добавляем элементы в дерево
Выводим дерево
    __50___   
   /       \  
  30_     70_ 
 /   \   /   \
20  40  60  80

Удаляем элемент
  __50___   
 /       \  
30_     70_ 
   \   /   \
  40  60  80

Поиск элемента
40


#### 1) Генерация дерева
Случайным образом сгенерировать 25 чисел в диапазоне от 1 до 50.
Каждое вновь сгенерированное число использовать в качестве ключа при добавлении в ДДП.
Вывести на экран получившееся дерево (любым удобным для прочтения/интерпретации способом).

In [ ]:
def gen_and_drow_tree(node_amnt, min_val, max_val):
    bst = BinarySearchTree()

    for t in range(node_amnt):
        bst.insert(randint(min_val, max_val))

    bst.display()
    return bst


bst = gen_and_drow_tree(25, 1, 50)

  ______12_                                 
 /         \                                
 2        12_____________________________   
/ \                                      \  
1 2__               ____________________46_ 
     \             /                       \
    _9_       ____24_                     49
   /   \     /       \                      
   2  10    13___   24_____________         
    \            \                 \        
    2           23      __________38___     
               /       /               \    
              22      24_____         43    
                             \       /      
                          __30_     40      
                         /     \            
                        24_   30_           
                           \     \          
                          26    33          


#### 2) Получение массива ключей
Из получившегося дерева получить отсортированный массив ключей размера 25.

In [ ]:
def get_sorted_keys(bst):
    lst = []
    cur_node = bst.min_value_node(bst.root)
    stop_node = bst.max_value_node(bst.root)

    while cur_node != stop_node:
        lst.append(cur_node.val)

        if cur_node.right is None:

            while True:

                if cur_node.parent.left is not None and cur_node.parent.left == cur_node:
                    cur_node = cur_node.parent
                    break

                cur_node = cur_node.parent

        else:
            cur_node = bst.min_value_node(cur_node.right)

    else:
        lst.append(stop_node.val)

    return lst

print(get_sorted_keys(bst))

[1, 2, 2, 2, 2, 9, 10, 12, 12, 13, 22, 23, 24, 24, 24, 24, 26, 30, 30, 33, 38, 40, 43, 46, 49]


#### 3) Выполнение п. 1-2 с большими данными и замером времени
Повторить пункты 1-2 для 1.000/5.000/10.000 чисел в диапазоне:
a. от 1 до 10.000,
b. от 1 до 500
при этом необходимо произвести замер времени.

##### функция для замера времени

In [ ]:
def count_time(func, *args, **kwargs):
    """
    func - исследуемая функция
    *args, **kwargs - параметры, передаваемые в функцию
    unit_of_measure - единицы измерения времени (по дефолту стоят микросекунды):
        s - секунда
        ms - миллисекунда
        us - микросекунда
    """
    try:
        unit_of_measure = kwargs['unit_of_measure']
        del kwargs['unit_of_measure']
    except KeyError:
        unit_of_measure = 'us'

    time_start = time.perf_counter()
    res = func(*args, **kwargs)
    time_finish = time.perf_counter()

    time_s = (time_finish - time_start)

    if unit_of_measure == 's': return int(time_s)
    elif unit_of_measure == 'ms': return int(time_s*1000)
    else: return int(time_s*1000000)

##### создание таблицы для сохранения замеров времени

In [ ]:
results = pd.DataFrame(
    columns=[
        'node_amnt',
        'max_node_val',
        'bst_sort_time'
        ]
    )

In [ ]:
for node_amnt in (1_000, 5_000, 10_000):
    for max_node_val in (500, 10_000):
        print(f'Binary search tree with {node_amnt} nodes and node value in range [1, {max_node_val}]')
        bst = gen_and_drow_tree(node_amnt, 1, max_node_val)
        time_us = count_time(get_sorted_keys, bst)
        results.loc[len(results.index)] = [node_amnt, max_node_val, time_us]


Binary search tree with 1000 nodes and node value in range [1, 500]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#### 4) Генерация массивов с последующей сортировкой и замером времени
Сгенерировать массивы чисел размера 1.000/5.000/10.000 (сначала в диапазоне от 1 до 10.000, потом в диапазоне от 1 до 500).
Произвести сортировку массивов с использованием любого алгоритма сортировки (используйте библиотечные функции python), при это не забудьте произвести замеры времени работы на каждом размере массива.


In [ ]:
def gen_lst(el_amnt, min_val, max_val):
    return [randint(min_val, max_val) in range(el_amnt)]


def sort_lst(lst):
    sort_lst = lst.sort()
    return sort_lst


lst_sort_time = []

for el_amnt in (1_000, 5_000, 10_000):
    for max_val in (500, 10_000):
        lst = gen_lst(el_amnt, 1, max_val)
        time_us = count_time(sort_lst, lst)
        lst_sort_time.append(time_us)


results['lst_sort_time'] = lst_sort_time

#### 5) Выводы
Всю полученную информацию внесите в таблицу и сделайте выводы.

##### таблица со всей полученной информацией

In [ ]:
results

,node_amnt,max_node_val,bst_sort_time,lst_sort_time
0,1000,500,635,3
1,1000,10000,421,1
2,5000,500,2506,1
3,5000,10000,1969,1
4,10000,500,8826,0
5,10000,10000,5073,1


##### вывод
Из полученных данных можно сделать вывод, что двоичное дерево поиска в нашей реализации не рекомендуется к использованию в качестве вспомогательного средства при сортировке данных.

# 2 часть.
### Цель задания – познакомиться со структурой данных ДДП.

### 1. Определение высоты дерева
Напишите функцию, определяющую высоту дерева. Продемонстрируйте работу этой функции.

**Высота двоичного дерева** — это количество ветвей между его корнем и наиболее глубоко расположенным листовым узлом (этот лист лежит на самом нижнем уровне). Связи между узлами дерева называют ветвями.

Если бинарное поисковое дерево пустое, то есть не содержит ни одного элемента, то его высота = −1
<br>
Если есть только корень, то высота = 0
<br>
<br>
Сложность этого алгоритма O(n), где n - количество узлов

In [ ]:
def tree_height(node):
    if node is None:
        return -1
    else:
        left_height = tree_height(node.left)
        right_height = tree_height(node.right)

        #Высота дерева на 1 больше максимальной высоты из поддеревьев
        return max(left_height, right_height) + 1

In [ ]:
#демонстрация работы функции

bst = BinarySearchTree()

print("Добавляем элементы в дерево")
bst.insert(10)
bst.insert(50)
bst.insert(100)
bst.insert(40)
bst.insert(70)
bst.insert(60)
bst.insert(80)

print("Выводим дерево\n")
bst.display()

height = tree_height(bst.root)
print("\nВысота дерева:", height)

Добавляем элементы в дерево
Выводим дерево

10___          
     \         
    50_______  
   /         \ 
  40      __100
         /     
        70_    
       /   \   
      60  80   

Высота дерева: 4


### 2.	Удаление вершин заданного значения с последующим выводом дерева
Сгенерируйте случайным образом ДДП, состоящее из 50 узлов, содержащих ключи в диапазоне от 1 до 25. Далее пользователь вводит любое число X. В построенном дереве производится удаление всех вершин, у которых ключ равен X, и вывод получившегося дерева, либо пользователю сообщается, что вершины с данным ключом X в дереве не существует.

In [ ]:
def generate_tree():
    tree = BinarySearchTree()
    keys = [randint(1, 25) for _ in range(50)]
    for key in keys:
        tree.insert(key)
    return tree

In [ ]:
tree = generate_tree()
print("Исходное дерево:")
tree.display()

# Ввод пользователем числа X
X = int(input("Введите число X для удаления: "))
if tree.search(X) is not None:

    while tree.search(X) is not None:
        tree.delete(X)

    print(f"Узлы с ключом {X} удалены. Дерево после удаления:")
    tree.display()

else:
    print(f"Узлы с ключом {X} в дереве не существуют.")

Исходное дерево:
 __2__                                                                         
/     \                                                                        
1    _3_______________________________________________________                 
 \  /                                                         \                
 1  2             ___________________________________________21_               
  \  \           /                                              \              
  1  2         __8____________                                 21_             
              /               \                                   \            
              6            __13_                                 21_________   
             / \          /     \                                           \  
          ___5 6     ____12_   13_                                       __24_ 
         /      \   /       \     \                                     /     \
        _4      6  _9_ 

# черновик